In [1]:
%matplotlib inline
from __future__ import division, print_function
import os

In [2]:
!ls

Analyze Motif Dist In Sim.ipynb
compute_imp_scores.py
compute_imp_scores.pyc
compute_imp_scores.sh
config
dan_basset_keras_port
dinucshuff_motifs_inserted_sigthresh5e-6_sorted_naive_window_around_summit.bed.gz
imp_scores
insert_motifs_in_dincuc_shuff_regions.py
model_files
motif_hits_sorted_naive_window_around_summit.bed.gz
nonoverlap_dinucshuff_motifs_inserted_sigthresh5e-6_merged_universal_neg_representative_peaks.bed.gz
runs_perf-metric-auROC.db
sorted_naive_window_around_summit.bed.gz
sorted_naive_window_around_summit.fa.gz
Study Simulated Importance.ipynb
test_set_negatives.bed.gz
test_set_positives.bed.gz
train_dinucshuff_motifs_inserted_sigthresh5e-6_sorted_naive_window_around_summit.bed.gz
train_nonoverlap_dinucshuff_motifs_inserted_sigthresh5e-6_merged_universal_neg_representative_peaks.bed.gz
train_sim_model_from_preinit.sh
valid_labels.gz


In [3]:
!zcat test_set_positives.bed.gz | wc -l

18421


In [4]:
import gzip
from collections import namedtuple

MotifHit = namedtuple("MotifHit",['offset', 'motif_id',
                                  'motif_factor', 'motif_seq',
                                  'motif_strand', 'motif_sig'])
def parse_motif_hit(string):
    (offset,motif_id,motif_factor,
     motif_seq,motif_strand,motif_sig) = string.split(",")
    
    offset = int(offset)
    motif_sig = float(motif_sig)
    return MotifHit(offset=offset,
                    motif_id=motif_id,
                    motif_factor=motif_factor,
                    motif_seq=motif_seq,
                    motif_strand=motif_strand,
                    motif_sig=motif_sig)

def parse_sim_file(file_name):
    seqs = []
    motifs_arr = []
    seq_ids = []
    
    for line in gzip.open(file_name):
        line = line.rstrip("\n")
        chrom,start,end,seq_id,seq,motifs = line.split("\t")
        seqs.append(seq)
        motifs_arr.append([parse_motif_hit(x) for x in motifs.split(";")] if len(motifs) > 0 else [])
        seq_ids.append(seq_id)
    return seqs, motifs_arr, seq_ids

original_positives_seqs, original_positives_motifs, original_positives_ids = parse_sim_file("test_set_positives.bed.gz")
negatives_seqs, negatives_motifs, negatives_ids = parse_sim_file("test_set_negatives.bed.gz")

In [5]:
positives_seqs = original_positives_seqs[::10]
positives_motifs = original_positives_motifs[::10]
positives_ids = original_positives_ids[::10]

In [39]:
from collections import Counter
original_positives_motif_counts_in_central_300 = Counter([x.motif_factor+":"+x.motif_id for y in original_positives_motifs for x in y
                                       if (x.offset > 350 and (x.offset + len(x.motif_seq))<650)])
original_positives_motif_counts_outside_central_300 = Counter([x.motif_factor+":"+x.motif_id for y in original_positives_motifs for x in y
                                                      if (x.offset > 650 or (x.offset + len(x.motif_seq))<350)])
negatives_motif_counts_in_central_300 = Counter([x.motif_factor+":"+x.motif_id for y in negatives_motifs for x in y
                                       if (x.offset > 350 and (x.offset + len(x.motif_seq))<650)])
negatives_motif_counts_outside_central_300 = Counter([x.motif_factor+":"+x.motif_id for y in negatives_motifs for x in y
                                                      if (x.offset > 650 or (x.offset + len(x.motif_seq))<350)])


In [54]:
ratios_in_to_out_negatives = dict([(x,((negatives_motif_counts_in_central_300[x]+1)
                             *(7/3)/(negatives_motif_counts_outside_central_300[x]+1),
                   negatives_motif_counts_in_central_300[x]))
                   for x in negatives_motif_counts_in_central_300.keys()])
ratios_in_to_out_positives = dict([(x,((original_positives_motif_counts_in_central_300[x]+1)
                                       *(7/3)/(original_positives_motif_counts_outside_central_300[x]+1),
                                    original_positives_motif_counts_in_central_300[x],
                                    ratios_in_to_out_negatives[x]))
                                    for x in original_positives_motif_counts_in_central_300.keys()])
ratios_positives_to_negatives = dict([(x,((original_positives_motif_counts_in_central_300[x])
                                       *(len(negatives_seqs)/len(original_positives_seqs))/(negatives_motif_counts_in_central_300[x]),
                                       original_positives_motif_counts_in_central_300[x]))
                                    for x in original_positives_motif_counts_in_central_300.keys()])

sorted_motif_list = [(x, ratios_positives_to_negatives[x][0],
        ratios_in_to_out_positives[x][0],
        ratios_in_to_out_negatives[x][0])
 for x,y in sorted(ratios_in_to_out_positives.items(), key=lambda x: -x[1][0])
 if (ratios_positives_to_negatives[x][0] > 1.5
     and ratios_in_to_out_positives[x][0] > 1.5
     and (ratios_in_to_out_positives[x][0] > ratios_in_to_out_negatives[x][0]*1.5))]
motif_ids_to_assess = [x[0] for x in sorted_motif_list]
print(motif_ids_to_assess)
motif_ids_to_assess_set = set(motif_ids_to_assess)

['CREB3:M5323_1.02', 'HNF1B:M6282_1.02', 'HNF4A:M4698_1.02', 'HNF4G:M2286_1.02', 'DMBX1:M5345_1.02', 'FOXA2:M4567_1.02', 'HNF1A:M6281_1.02', 'XBP1:M6536_1.02', 'NR2F1:M6176_1.02', 'RXRG:M6462_1.02', 'NR1H2:M6383_1.02', 'HIF1A:M6275_1.02', '(SP5)_(Drosophila_melanogaster)_(DBD_0.93):M5209_1.02', 'TCF7L1:M5903_1.02', 'NR2F6:M6392_1.02', '(KLF12)_(Mus_musculus)_(DBD_1.00):M0443_1.02', '(KLF7)_(Mus_musculus)_(DBD_1.00):M0405_1.02', '(ZFP161)_(Mus_musculus)_(DBD_1.00):M0432_1.02', 'NR6A1:M6397_1.02', 'E2F4:M4537_1.02']


In [104]:
from collections import defaultdict

motif_to_hit_locations = defaultdict(list)
motif_to_length = {}

for region_idx,motifs_arr in enumerate(positives_motifs):
    for motif in motifs_arr:
        if (motif.offset > 350 and (motif.offset + len(motif.motif_seq)) < 650):
            motif_id = motif.motif_factor+":"+motif.motif_id
            if (motif_id in motif_ids_to_assess):
                motif_to_hit_locations[motif_id].append((region_idx, motif.offset-350))
                if (motif_id in motif_to_length):
                    assert motif_to_length[motif_id] == len(motif.motif_seq)
                else:
                    motif_to_length[motif_id] = len(motif.motif_seq)

all_motif_lengths = sorted(set(motif_to_length.values()))


In [114]:
import numpy as np

negatives_subsample_factor = 100

#prepare negative windows
motif_length_to_negatives_locations = defaultdict(list)
val_lookup = {'a': 0, 'c': 0, 'g': 0, 't': 0, 'n':0, 'A': 1, 'C': 1, 'G':1, 'T':1, 'N':1}

for i,sequence in enumerate(positives_seqs):
    arr = np.array([val_lookup[x] for x in sequence])
    cumsum_arr = np.array([0]+list(np.cumsum(arr)))
    for motif_length in all_motif_lengths:
        windows_sums = cumsum_arr[motif_length:]-cumsum_arr[:-motif_length]
        null_windows = [(i,x-350) for x in np.nonzero(windows_sums==0)[0][::negatives_subsample_factor]
                        if x > 350 and x < (650-motif_length)]
        motif_length_to_negatives_locations[motif_length].extend(null_windows)


In [117]:
import h5py
import numpy as np
from sklearn.metrics import average_precision_score
import sys

model_ids = ["record_1_model_Ba1Th",
             "record_2_model_fijFp",
             "record_3_model_iNu96",
             "record_4_model_s3EVY",
             "record_5_model_k1Ccu",
             "record_6_model_X79UW",
             "record_7_model_XpjQJ",
             "record_8_model_XchY4",
             "record_9_model_8fTYg",
             "record_10_model_xpYJA"]

scoring_methods = ['scores_ism',
'scores_deeplift_rescale_dinucshuff',
'scores_deeplift_genomicsdefault_dinucshuff',
'scores_integrated_grad20_dinucshuff',
'scores_grad_times_inp_flatref',

'scores_integrated_grad20_avgposref',
'scores_deeplift_rescale_avgposref',
'scores_deeplift_genomicsdefault_avgposref',
'scores_integrated_grad20_flatref',
'scores_deeplift_rescale_flatref',
'scores_deeplift_genomicsdefault_flatref',

'scores_integrated_grad10_dinucshuff',
'scores_integrated_grad2_dinucshuff',
'scores_integrated_grad5_dinucshuff']

model_id_to_scoring_method_to_motif_to_auprc = defaultdict(lambda: defaultdict(dict))
model_id_to_scoring_method_to_motif_to_auprcoverbg = defaultdict(lambda: defaultdict(dict))

for model_id in model_ids:
    f = h5py.File("oneinten_imp_scores/imp_scores_"+model_id+".h5", "r")
    print("on",model_id)
    sys.stdout.flush()
    for scoring_method in scoring_methods:
        sys.stdout.flush()
        scores = np.array(f[scoring_method])
        negatives_locations_scores = {}
        for motif_length in motif_length_to_negatives_locations:
            negatives_locations_scores[motif_length] = [
                np.sum(scores[x[0],x[1]:x[1]+motif_length]) for x in
                motif_length_to_negatives_locations[motif_length]]
        motif_to_hit_locations_scores = {}
        motif_to_auprc = {}
        motif_to_auprcoverbg = {}
        for motif in motif_to_hit_locations:
            motif_length = motif_to_length[motif]
            hit_locations_scores = [
                np.sum(scores[x[0], x[1]:x[1]+motif_length]) for x in
                motif_to_hit_locations[motif]]
            motif_to_hit_locations_scores[motif] = hit_locations_scores
            y_true = ([1 for x in hit_locations_scores]
                      + [0 for x in negatives_locations_scores[motif_length]])
            y_score = hit_locations_scores + negatives_locations_scores[motif_length]
            auprc = average_precision_score(y_true=y_true,
                                            y_score=y_score)
            motif_to_auprc[motif] = auprc
            motif_to_auprcoverbg[motif] = auprc/np.mean(y_true)
            
            model_id_to_scoring_method_to_motif_to_auprc[model_id][scoring_method] = motif_to_auprc
            model_id_to_scoring_method_to_motif_to_auprcoverbg[model_id][scoring_method] = motif_to_auprcoverbg


on record_1_model_Ba1Th
scores_ism
scores_deeplift_rescale_dinucshuff
scores_deeplift_genomicsdefault_dinucshuff
scores_integrated_grad20_dinucshuff
scores_grad_times_inp_flatref
scores_integrated_grad20_avgposref
scores_deeplift_rescale_avgposref
scores_deeplift_genomicsdefault_avgposref
scores_integrated_grad20_flatref
scores_deeplift_rescale_flatref
scores_deeplift_genomicsdefault_flatref
scores_integrated_grad10_dinucshuff
scores_integrated_grad2_dinucshuff
scores_integrated_grad5_dinucshuff
on record_2_model_fijFp
scores_ism
scores_deeplift_rescale_dinucshuff
scores_deeplift_genomicsdefault_dinucshuff
scores_integrated_grad20_dinucshuff
scores_grad_times_inp_flatref
scores_integrated_grad20_avgposref
scores_deeplift_rescale_avgposref
scores_deeplift_genomicsdefault_avgposref
scores_integrated_grad20_flatref
scores_deeplift_rescale_flatref
scores_deeplift_genomicsdefault_flatref
scores_integrated_grad10_dinucshuff
scores_integrated_grad2_dinucshuff
scores_integrated_grad5_dinucshuf

In [133]:
motif_to_hit_locations['HNF4A:M4698_1.02']

[(63, 148),
 (67, 206),
 (69, 116),
 (84, 200),
 (173, 241),
 (202, 127),
 (233, 191),
 (252, 178),
 (278, 183),
 (301, 242),
 (361, 189),
 (366, 178),
 (376, 118),
 (402, 119),
 (414, 44),
 (446, 118),
 (471, 177),
 (506, 150),
 (513, 73),
 (520, 184),
 (531, 98),
 (582, 164),
 (638, 131),
 (641, 12),
 (714, 117),
 (718, 147),
 (783, 50),
 (812, 80),
 (812, 250),
 (899, 104),
 (904, 202),
 (919, 46),
 (919, 199),
 (951, 136),
 (986, 210),
 (1016, 122),
 (1026, 163),
 (1089, 93),
 (1115, 105),
 (1154, 66),
 (1161, 33),
 (1177, 262),
 (1241, 214),
 (1250, 58),
 (1305, 80),
 (1324, 138),
 (1342, 29),
 (1367, 137),
 (1371, 191),
 (1384, 149),
 (1387, 157),
 (1397, 108),
 (1403, 139),
 (1416, 41),
 (1454, 177),
 (1497, 216),
 (1499, 2),
 (1500, 43),
 (1534, 199),
 (1542, 165),
 (1570, 133),
 (1580, 133),
 (1588, 136),
 (1596, 66),
 (1608, 193),
 (1636, 47),
 (1644, 213),
 (1672, 130),
 (1682, 134),
 (1722, 124),
 (1790, 68),
 (1794, 199),
 (1821, 108),
 (1821, 248),
 (1831, 114)]

In [132]:
selected_motif = 'HNF4A:M4698_1.02'
for scoring_method in scoring_methods:
    print(scoring_method)
    for model_id in model_ids:
        print(model_id_to_scoring_method_to_motif_to_auprcoverbg
              [model_id][scoring_method][selected_motif])

scores_ism
1.2689155889189399
1.3343313055246253
1.217241136848713
1.503974465817277
1.5070533960981467
1.3295317110831404
1.439583364493182
1.7885512851744467
1.1513774344504297
1.6205393463153044
1.6205393463153044
scores_deeplift_rescale_dinucshuff
0.8420725184610258
0.9102645052701098
0.8800000615920844
0.7978694643945993
0.8892114265306029
0.8294083583885741
0.9158087779801339
0.8011729563502908
1.0613025554967943
0.7878362546431112
0.7878362546431112
scores_deeplift_genomicsdefault_dinucshuff
0.7749932575197911
0.9100610481398917
0.8636172835346743
0.7651796532296119
0.8478112610224292
0.7854704743554497
0.879473652134581
0.821651530723346
1.0061820550681926
0.7918344506208119
0.7918344506208119
scores_integrated_grad20_dinucshuff
1.0054222058053253
1.078801340014907
1.1782131020691649
0.9502305564937377
1.1410161267257894
0.9504784894885823
1.1530318456252164
0.9295475584315995
1.0409737484498744
1.1058909651107363
1.1058909651107363
scores_grad_times_inp_flatref
0.8292596110775

In [128]:
sorted([(x,len(y)) for x,y in motif_to_hit_locations.items()],
        key=lambda x: -x[1])

[('(SP5)_(Drosophila_melanogaster)_(DBD_0.93):M5209_1.02', 168),
 ('(KLF7)_(Mus_musculus)_(DBD_1.00):M0405_1.02', 155),
 ('(KLF12)_(Mus_musculus)_(DBD_1.00):M0443_1.02', 152),
 ('HNF4A:M4698_1.02', 75),
 ('HNF4G:M2286_1.02', 70),
 ('NR2F1:M6176_1.02', 43),
 ('(ZFP161)_(Mus_musculus)_(DBD_1.00):M0432_1.02', 38),
 ('HNF1A:M6281_1.02', 24),
 ('NR2F6:M6392_1.02', 24),
 ('TCF7L1:M5903_1.02', 21),
 ('RXRG:M6462_1.02', 20),
 ('FOXA2:M4567_1.02', 19),
 ('NR6A1:M6397_1.02', 19),
 ('NR1H2:M6383_1.02', 17),
 ('E2F4:M4537_1.02', 17),
 ('HNF1B:M6282_1.02', 14),
 ('CREB3:M5323_1.02', 9),
 ('HIF1A:M6275_1.02', 7),
 ('XBP1:M6536_1.02', 6),
 ('DMBX1:M5345_1.02', 4)]

In [ ]:
['CREB3:M5323_1.02', 'HNF1B:M6282_1.02', 'HNF4A:M4698_1.02', 'HNF4G:M2286_1.02', 'DMBX1:M5345_1.02', 'FOXA2:M4567_1.02', 'HNF1A:M6281_1.02', 'XBP1:M6536_1.02', 'NR2F1:M6176_1.02', 'RXRG:M6462_1.02', 'NR1H2:M6383_1.02', 'HIF1A:M6275_1.02', '(SP5)_(Drosophila_melanogaster)_(DBD_0.93):M5209_1.02', 'TCF7L1:M5903_1.02', 'NR2F6:M6392_1.02', '(KLF12)_(Mus_musculus)_(DBD_1.00):M0443_1.02', '(KLF7)_(Mus_musculus)_(DBD_1.00):M0405_1.02', '(ZFP161)_(Mus_musculus)_(DBD_1.00):M0432_1.02', 'NR6A1:M6397_1.02', 'E2F4:M4537_1.02']